# G5HT-PIPELINE

## TODO:

1. I wonder if I computed a spline on each and every z slice and warped each, oriented each of them, and warped each of them, if the problem of weirdly sheared image stacks would be solved
2. quick mp4 for all recordings
   1. now working in engaging, works per one nd2 sbatch
3. focus check for all recordings
   1. maybe focus check can be used to specify which z slices are good to use and which frames are good to use
4. for recordings starting in december 2025, need to trim first 2 rather than last 2 z slices
5. flip worms so that VNC is always up
6. fixed mask could be automated, but if not, make sure to save which index is fixed
7. extract behavior
8. posture similarity
   1. posture might consist of the spline + thresholded z-stack
      1. I'm thinking that the orientation shouldn't matter, but the z-planes in focus will, and curvature/spline of the head will
      2. maybe need to actually interpolate to 117 z slices
   2. sub registration problems
   3. label each set of registered frames with one set of ROIs, or auto segment ROIs from each set of registered frames
9.  track z over time, which zslices are consistent
   1. focus + correlation
10. beads -> train/test
11. gfp+1 relative to rfp channel (might only apply to pre december 2025 recordings)
12. wholistic 
    1.  parameter sweep, might change
    2.  python version
    3.  actually, wholistic might be tricky to use all the time, because it only works after parameter optimization, which I don't really know how to automate
13. autocorr/scorr
14. automate z slice trimming
    1.  pre december 2025 (trim last 2 z slices)
    2.  post december 2025 (trim first z slice)
15. photobleaching estimation?
    1.  record immo with serotonin
    2.  at least do it for RFP
16. try deltaF/F [ (F(t) - F0) / F0 ]
18. maybe remove right-most part of the worm where spline is usually kinked
19. port everything to engaging

## CONDA ENVIRONMENTS

For steps __1. preprocess__ and __2. mip__, `conda activate g5ht-pipeline`

For step __3. segment__, `conda activate segment-torch` or `conda activate torchcu129`

For step __4. spline, 5. orient, 6. warp, 7. reg__


## IMPORTS

In [1]:
import sys
import os
import importlib
from tqdm import tqdm

try:
    import utils
    is_torch_env = False
except ImportError:
    is_torch_env = True
    print("utils not loaded because conda environment doesn't have nd2reader installed. probably using torchcu129 env, which is totally fine for just doing the segmentation step")

## SPECIFY DATA TO PROCESS

In [ ]:
DATA_PTH = r'D:\DATA'
PROJECT = 'g5ht-free' # g5ht-free or g5ht-immo
DATE = '20251223'

DATA_PTH = os.path.join(DATA_PTH, PROJECT, DATE)

INPUT_ND2 = f'date-20251223_strain-ISg5HT_condition-starvedpatch_worm005.nd2'

INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)

NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise\noise_111125.tif'



OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41 if 'immo' not in INPUT_ND2 else 122

if not is_torch_env:
    noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)
    num_frames, height, width, num_channels = utils.get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 
    # num_frames = 1200
    # height = 512
    # width = 512
    # num_channels = 2
    beads_alignment_file = utils.get_beads_alignment_file(INPUT_ND2_PTH)
else:
    print("utils not loaded because conda environment doesn't have nd2reader installed. probably using torchcu129 env, which is totally fine for just doing the segmentation step")

print(INPUT_ND2)
print('Num z-slices: ', STACK_LENGTH)
if not is_torch_env:
    print('Number of frames: ', num_frames)
    print('Height: ', height)
    print('width: ', width)
    print('Number of channels: ', num_channels)
    print('Beads alignment file: ', beads_alignment_file)

date-20251223_strain-ISg5HT_condition-starvedpatch_worm005.nd2
Num z-slices:  41
Number of frames:  1200
Height:  512
width:  512
Number of channels:  2
Beads alignment file:  D:\DATA\g5ht-free\20251223\date-20251223_strain-ISg5HT_condition-starvedpatch_worm005_chan_alignment.nd2


## 0. PROCESS BEADS ALIGNMENT DATA (OPTIONAL, CHANGING THIS SO BEADS ARE PROCESSED SEAMLESSLY IN PIPELINE)

` conda activate g5ht-pipeline`

The registration parameters between green and red channels will be applied to worm recordings

### SHEAR CORRECT AND CHANNEL REGISTER

In [ ]:
from preprocess_parallel import main as preprocess_nd2_parallel
_ = importlib.reload(sys.modules['preprocess_parallel'])

num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 

# # command-line arguments
sys.argv = ["", beads_alignment_file, "0", str(num_frames_beads-1), NOISE_PTH, STACK_LENGTH, 5, num_frames_beads, height, width, num_channels]

# # Call the main function
preprocess_nd2_parallel()

### MIP

This step saved the median channel registration parameters, need to do this somewhere else

In [ ]:
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", beads_alignment_file, STACK_LENGTH, num_frames_beads, 2]

# Call the main function
mip()

## 1. SHEAR CORRECTION

` conda activate g5ht-pipeline`

- shear corrects each volume
  - depending on each exposure time, it can take roughly half a second between the first and last frames of a volume, so any movements need to be corrected for
- creates one `.tif` for each volume and stores it in the `shear_corrected` directory

##### TODO: should probably update stack length after shear correction since we cut it by 2, although not sure it's explicitly needed

In [ ]:
import shear_correct
_ = importlib.reload(sys.modules['shear_correct'])

# start_index = "0"
# end_index = str(num_frames-1)
start_index = "100"
end_index = "110"
cpu_count = str(int(os.cpu_count() / 2))
# cpu_count = str(int(os.cpu_count()))
skip_shear_correction = True # if True, will just denoise and save as tif

# sys.argv = ["", nd2 file, start_frame, end_frame, noise_pth, stack_length, n_workers, num_frames, height, width, num_channels]
sys.argv = ["", INPUT_ND2_PTH, start_index, end_index, NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, z2keep, skip_shear_correction]

# Call the main function
shear_correct.main()

Processing 11 stacks (100-110) using 12 workers...


  0%|          | 0/11 [00:18<?, ?it/s]


TypeError: cannot unpack non-iterable bool object

## 2. CHANNEL ALIGNMENT

` conda activate g5ht-pipeline`

### 2a. GET MEDIAN CHANNEL ALIGNMENT PARAMETERS FROM ALL FRAMES

- If channel alignment file found, uses that, if not uses worm recording
- creates a `.txt` file for each volume that contains elastix channel registration parameters
- creates `chan_align_params.csv` and  `chan_align.txt`

In [ ]:
beads_alignment_file

In [ ]:
import get_channel_alignment
import median_channel_alignment
_ = importlib.reload(sys.modules['get_channel_alignment'])
_ = importlib.reload(sys.modules['median_channel_alignment'])

## set beads_alignment_file to None to use worm recording for channel alignment, even if beads file exists
# beads_alignment_file = None

start_index = "0"
# cpu_count = str(int(os.cpu_count() / 2))
cpu_count = str(int(os.cpu_count()))

if beads_alignment_file is not None:
    align_with_beads = True
    num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 
    sys.argv = ["", beads_alignment_file, start_index, str(num_frames_beads-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames_beads, height, width, num_channels, align_with_beads]
else:
    align_with_beads = False
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads]

# # Call the main function
get_channel_alignment.main()
median_channel_alignment.main()


### 2b. APPLY MEDIAN CHANNEL ALIGNMENT PARAMETERS

- ouputs aligned volumes in `channel_aligned` directory

In [ ]:
import apply_channel_alignment
_ = importlib.reload(sys.modules['apply_channel_alignment'])

start_index = "0"
# cpu_count = str(int(os.cpu_count() / 2))
cpu_count = str(int(os.cpu_count()))

# 0786 to 0799 are bad frames in worm005.nd2, copied 0785 for each of those frames

if beads_alignment_file is not None:
    align_with_beads = True
    num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads, beads_alignment_file]
else:
    align_with_beads = False
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads]


# Call the main function
apply_channel_alignment.main()

In [ ]:
# # create copies of 0785 and rename it to 0786 to 0799
# import shutil
# for i in range(786, 800):
#     shutil.copyfile(r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251223\date-20251223_strain-ISg5HT_condition-starvedpatch_worm005\channel_aligned\0785.tif',
#                     r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251223\date-20251223_strain-ISg5HT_condition-starvedpatch_worm005\channel_aligned\{:04d}.tif'.format(i))

### 2c. PLOT CHANNEL ALIGNMENT PARAMETER DISTRIBUTIONS

In [ ]:
out_dir

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# make font sizes larger for visibility
plt.rcParams.update({'font.size': 18})

try:
    out_dir = os.path.splitext(INPUT_ND2_PTH)[0]

    df = pd.read_csv(os.path.join(out_dir, 'chan_align_params.csv'))
    params = ['TransformParameter_0', 'TransformParameter_1', 'TransformParameter_2', 'TransformParameter_3', 'TransformParameter_4', 'TransformParameter_5']
    labels = ['Rx', 'Ry', 'Rz', 'Tx', 'Ty', 'Tz']

    # the xaxis limits for each subplot should be the same across figures

    xlims = np.zeros((6,2))

    plt.figure(figsize=(12,8), tight_layout=True)
    for i,param in enumerate(params):
        plt.subplot(2, 3, i+1)
        plt.hist(df[param], bins=30, color='red', alpha=0.6)
        # plot the median value as a vertical line
        median_value = df[param].median()
        plt.axvline(median_value, color='black', linestyle='dashed', linewidth=2)
        plt.xlabel(labels[i])
        plt.ylabel('Frequency')
        # get xaxis limits
        xlims[i,:] = plt.xlim()
        # title is median value
        plt.title(f'Median: {np.round(median_value,3)}', fontsize=14)
    plt.show()
except FileNotFoundError:
    print("No chan_align_params.csv found for worm recording")

out_dir = os.path.splitext(INPUT_ND2_PTH)[0] + '_chan_alignment'
df = pd.read_csv(os.path.join(out_dir, 'chan_align_params.csv'))
params = ['TransformParameter_0', 'TransformParameter_1', 'TransformParameter_2', 'TransformParameter_3', 'TransformParameter_4', 'TransformParameter_5']
labels = ['Rx', 'Ry', 'Rz', 'Tx', 'Ty', 'Tz']

plt.figure(figsize=(12,8), tight_layout=True)
for i,param in enumerate(params):
    plt.subplot(2, 3, i+1)
    plt.hist(df[param], bins=30, color='blue', alpha=0.6)
    # plot the median value as a vertical line
    median_value = df[param].median()
    plt.axvline(median_value, color='black', linestyle='dashed', linewidth=2)
    plt.xlabel(labels[i])
    plt.ylabel('Frequency')
    # apply xlims
    # plt.xlim(xlims[i,0], xlims[i,1])
    # title is median value, font size 14
    plt.title(f'Median: {np.round(median_value,3)}', fontsize=14)
plt.show()

## 3. BLEACH CORRECTION

TODO:
- per z slice?



In [ ]:
import importlib
import os
import sys

import bleach_correct
_ = importlib.reload(sys.modules['bleach_correct'])


PTH = os.path.splitext(INPUT_ND2_PTH)[0]
REG_DIR = 'channel_aligned' # 'channel_aligned' or 'tif' 
channels = 1
method = 'block' # 'block' or 'exponential'
mode = 'total' # 'total' or 'median'

bleach_correct.correct_bleaching(os.path.join(PTH,REG_DIR), channels=channels, method=method, fbc=0.04, intensity_mode=mode)


# # Correct RFP only with block method (default)
# correct_bleaching("path/to/data")

# # Correct both channels with exponential fit
# correct_bleaching("path/to/data", channels=[0, 1], method='exponential')

# # Command line
# python bleach_correct.py path/to/data --channels 0 1 --method exponential

## 4. MIP

` conda activate g5ht-pipeline`

- outputs `means.png`, `focus.png`, `mip.tif`, and `mip.mp4`, `focus_check.csv`

##### TODO: 
- legend for focus.png, should be frame#
- mip for xy, xz, zy
- mip for several slices

In [ ]:
import mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
framerate = 8
# tif_dir = 'bleach_corrected_RFP_block' # one of 'shear_corrected' 'channel_aligned' 'bleach_corrected_RFP_block'
tif_dir = 'shear_corrected'
rmax = 850
gmax = 150
mp4_quality = 10
do_focus = True
sys.argv = ["", INPUT_ND2_PTH, tif_dir, STACK_LENGTH, num_frames, framerate, rmax, gmax, mp4_quality, do_focus]

# Call the main function
mip.main()

## 5 DRIFT ESTIMATION

` conda activate g5ht-pipeline`

- outputs  `z_selection.csv`, `z_selection_diagnostics.png`, `sharpness.csv`

TODO:
- use z selection going forward
- also use sharpness/focus (and other things) to determine good/bad frames

In [ ]:
import drift_estimation

_ = importlib.reload(sys.modules['drift_estimation'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
tif_dir = 'bleach_corrected_RFP_block' # one of 'shear_corrected' 'channel_aligned' 'bleach_corrected_RFP_block'

sys.argv = ["", INPUT_ND2_PTH, tif_dir, STACK_LENGTH, num_frames]

# Call the main function
drift_estimation.main()

## 5. SEGMENT

- outputs `label.tif`, contains segmented MIP for each volume

__on home pc__: 
`conda activate segment-torch`

Uses a separate conda environment from the rest of the pipeline. create it using:
`conda env create -f segment_torch.yml`

__on lab pc__: 
`conda activate torchcu129`

Uses a separate conda environment from the rest of the pipeline. create it following steps in:
`segment_torch_cu129_environment.yml`

### setup each time model weights change
Need to set path to model weights as `CHECKPOINT` in `eval_torch.py`

In [ ]:
import segment.segment_torch
_ = importlib.reload(sys.modules['segment.segment_torch'])

# mip_tif = 'mip_bleach_corrected_RFP_block'
mip_tif = 'mip_channel_aligned' 

MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], f'{mip_tif}.tif')

# command-line arguments
sys.argv = ["", MIP_PTH]

segment.segment_torch.main()

## 6. SPLINE

`conda activate g5ht-pipeline`

- outputs `spline.json`, `spline.tif`, and `dilated.tif`

In [ ]:
import spline
_ = importlib.reload(sys.modules['spline'])

LABEL_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'label.tif')

# command-line arguments
sys.argv = ["", LABEL_PTH]

spline.main()

## 7. ORIENT

`conda activate g5ht-pipeline`

- outputs `oriented.json`, `oriented.png`, `oriented_stack.tif`

NOTE: `orient_v2.py` automated the process of finding orientation completely, whereas `orient.py` requires you to input the (x,y) nose location on the first frame

In [ ]:
import orient
_ = importlib.reload(sys.modules['orient'])

SPLINE_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'spline.json')
nose_y = 250
nose_x = 45

# apply constraints
# might need this when there are frames where the spline fitting fails and orientation is lost intermittently
constrain_frame = 515
constrain_frame_nose_y = 288
constrain_frame_nose_x = 180

# command-line arguments
# sys.argv = ["", SPLINE_PTH, str(nose_y), str(nose_x)]
sys.argv = ["", SPLINE_PTH, str(nose_y), str(nose_x), str(constrain_frame), str(constrain_frame_nose_y), str(constrain_frame_nose_x)]

orient.main()

In [ ]:
import orient_v2 # tried to automate finding nose point, not working well at the moment
_ = importlib.reload(sys.modules['orient_v2'])

SPLINE_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'spline.json')

# command-line arguments
sys.argv = ["", SPLINE_PTH]

orient_v2.main()

## 8. WARP

`conda activate g5ht-pipeline`

- ouputs: `warped/*.tif` and `masks/*.tif`

TODO: parallelize

In [ ]:
import warp
_ = importlib.reload(sys.modules['warp'])

PTH = os.path.splitext(INPUT_ND2_PTH)[0]

start_index = 516
end_index = num_frames

for i in tqdm(range(start_index, end_index)):
    # command-line arguments
    sys.argv = ["", PTH, i]

    warp.main()

## 9. REGISTER

`conda activate g5ht-pipeline`

__ALTERNATIVELY__: register using the wholistic registration algorithm, currently in MATLAB

TODO: parallelize / make faster

- pick a good representative fixed frame that you want to register everything to
  - copy it to the main output folder and name it `fixed_xxxx.tif`
  - copy the corresponding mask and name it `fixed_mask_xxxx.tif`

In [ ]:
import reg
_ = importlib.reload(sys.modules['reg'])

PTH = os.path.splitext(INPUT_ND2_PTH)[0]

start_index = 222
end_index = num_frames
zoom = 1 # albert was using 3
# zoom = 3

for i in tqdm(range(start_index, end_index)):
    # command-line arguments
    try:
        sys.argv = ["", PTH, i, str(zoom)]
        reg.main()
    except Exception as e:
        print(f"Error processing index {i}: {e}")   

### REGISTER WITH GFP+1 TO RFP

TRIM LAST RFP ZSLICE, TRIM FIRST GFP ZSLICE

seems to be that as of 20251204, all recordings were taken such that the i zslice in red channel corresponds to i+1 zslice in green channel

In [ ]:
import sys
import os
from tqdm import tqdm
import importlib

from reg_gfp_indexing import main as reg_worm

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned'

for i in tqdm(range(1200)):
    # command-line arguments
    sys.argv = ["", PTH, i, "1"]
    reg_worm()

### MAKE MOVIES OF REGISTERED DATA (see `reg_microfilm.ipynb`)

### REGISTER SINGLE FRAMES WITH ERROR LOGGING

In [ ]:
import tifffile
import numpy as np
import scipy.ndimage as ndi
import itk
import sys
import os
import glob

from itk import image_view_from_array

#get channels out of stacks
def register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack):
	fixed_rfp = fixed_stack[:, 1].copy()
	moving_gfp, moving_rfp = moving_stack[:, 0].copy(), moving_stack[:, 1].copy()

	#initialize registration parameters
	parameter_object = itk.ParameterObject.New()
	default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid', 4)
	parameter_object.AddParameterMap(default_rigid_parameter_map)
	default_affine_parameter_map = parameter_object.GetDefaultParameterMap('affine', 4)
	parameter_object.AddParameterMap(default_affine_parameter_map)
	default_bspline_128_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 128)
	parameter_object.AddParameterMap(default_bspline_128_parameter_map)
	default_bspline_64_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 64)
	parameter_object.AddParameterMap(default_bspline_64_parameter_map)
	default_bspline_32_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 32)
	parameter_object.AddParameterMap(default_bspline_32_parameter_map)

	#convert to itk images
	fixed_rfp = itk.image_view_from_array(fixed_rfp.astype(np.float32))
	moving_rfp = itk.image_view_from_array(moving_rfp.astype(np.float32))

	fixed_mask_stack = itk.image_view_from_array(fixed_mask_stack.astype(np.ubyte))
	moving_mask_stack = itk.image_view_from_array(moving_mask_stack.astype(np.ubyte))

	#register rfp first and then apply transform to gfp

	registered_rfp, transform_parameters = itk.elastix_registration_method(fixed_rfp, moving_rfp, parameter_object,
																		   fixed_mask=fixed_mask_stack, moving_mask=moving_mask_stack,
																		   log_to_console=True)
	registered_gfp = itk.transformix_filter(moving_gfp, transform_parameters)

	#initialize and fill output
	output_stack = np.zeros((fixed_stack.shape[0], 2, 200, 500), np.uint16)
	output_stack[:, 0] = np.clip(registered_gfp, 0, 4095)
	output_stack[:, 1] = np.clip(registered_rfp, 0, 4095)

	return output_stack
    
	# # enablle elastic error logging
	# elastix_filter = itk.ElastixRegistrationMethod.New(fixed_rfp, moving_rfp, parameter_object,
	# 																	   fixed_mask=fixed_mask_stack, moving_mask=moving_mask_stack,
	# 																	   log_to_console=True) 
    
	# elastix_filter.SetParameterObject(parameter_object)
	# elastix_filter.SetNumberOfThreads(8)
	# elastix_filter.LogToConsoleOn()  # Enable console logging
	# elastix_filter.LogToFileOn()
	# elastix_filter.SetOutputDirectory(r"C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\logs")
	# elastix_filter.Update()

	# return elastix_filter.GetOutput(), elastix_filter.GetTransformParameterObject()

input_dir = os.path.splitext(INPUT_ND2_PTH)[0]
warped_path = os.path.join(input_dir, 'warped')
output_path = os.path.join(input_dir, 'registered_fixed_sweep')
os.makedirs(output_path, exist_ok=True)

fixed_list = np.arange(0, 400, 50)
mov_list = np.arange(0, 400, 30)

for fixed in fixed_list:
    for mov in mov_list:
        print(f'Processing fixed: {fixed}, moving: {mov}')
        
		# load stacks
	
        moving_path = os.path.join(warped_path,f'{mov:04d}.tif')
        moving_stack = tifffile.imread(moving_path).astype(np.float32)
        fixed_path = os.path.join(warped_path,f'{fixed:04d}.tif')
        fixed_stack = tifffile.imread(fixed_path).astype(np.float32)
        fixed_mask_path = os.path.join(input_dir, 'masks', f'{fixed:04d}.tif')
        fixed_mask = tifffile.imread(fixed_mask_path)
        fixed_mask_stack = np.stack([fixed_mask] * fixed_stack.shape[0])

        moving_mask_path = os.path.join(input_dir, 'masks', f'{mov:04d}.tif')
        moving_mask = tifffile.imread(moving_mask_path)
        moving_mask_stack = np.stack([moving_mask] * fixed_stack.shape[0])


        output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)
        # save output stack with fixed and moving indices in filename
        output_file_path = os.path.join(output_path, f'fixed_{fixed:04d}_mov_{mov:04d}.tif')
        tifffile.imwrite(output_file_path, output_stack, imagej=True)

# fixed = 100
# mov = 200

# # load stacks
# moving_path = os.path.join(warped_path,f'{mov:04d}.tif')
# moving_stack = tifffile.imread(moving_path).astype(np.float32)
# fixed_path = os.path.join(warped_path,f'{fixed:04d}.tif')
# fixed_stack = tifffile.imread(fixed_path).astype(np.float32)
# fixed_mask_path = os.path.join(input_dir, 'masks', f'{fixed:04d}.tif')
# fixed_mask = tifffile.imread(fixed_mask_path)
# fixed_mask_stack = np.stack([fixed_mask] * fixed_stack.shape[0])

# moving_mask_path = os.path.join(input_dir, 'masks', f'{mov:04d}.tif')
# moving_mask = tifffile.imread(moving_mask_path)
# moving_mask_stack = np.stack([moving_mask] * fixed_stack.shape[0])


# output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)

In [ ]:
output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)

In [ ]:
output_stack.shape

In [ ]:

import matplotlib.pyplot as plt

plt.figure()
plt.pcolormesh(fixed_stack[10,1,:,:])
plt.show()

plt.figure()
plt.pcolormesh(output_stack[10,1,:,:])
plt.show()

## 10. EXTRACT BEHAVIOR

`conda activate imgpro` and probably conda/julia

- extract NIR video
- video viewer to label events (food encounter)
- extract behavior with antsun code
  - speed can be done now with stage position
- sync confocal and nir cams

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import hdf5plugin
import h5py


PTH = r'D:\DATA\g5ht-free\20251223'
FN = 'date-20251223_strain-ISg5HT_condition-starvedpatch_worm005'

f = h5py.File(os.path.join(PTH,FN + '.h5'), 'r')

im = f.get('img_nir')[:] # THW
sz = im.shape

fps = 20.0  # frames per second

print('Video shape: ', sz)

nframes = im.shape[0]
record_duration = nframes / fps # in seconds

print('Recording duration (s): ', record_duration)

### SAVE NIR MOVIE

In [ ]:
# save im, which is of shape (frame, height, width) as a .mp4 video
import cv2
out_fn = os.path.join(os.path.join(PTH,FN), 'nir_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(out_fn, fourcc, fps, (sz[2], sz[1]), isColor=False)
# save video
for i in range(nframes):
    # print status every 100 frames
    if i % 100 == 0:
        print('Saving frame: ', i, ' / ', nframes)
    frame = im[i,:,:]
    frame = (frame / np.max(frame) * 255).astype(np.uint8)
    # overlay frame with text of frame number and time in seconds
    time_sec = i / fps
    text = f'Frame: {i}  Time: {time_sec:.2f} s'
    cv2.putText(frame, text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255), 2)
    # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    out.write(frame)
out.release()
print('Saved video to: ', out_fn)


### SYNC CONFOCAL & NIR

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import hdf5plugin
import h5py


PTH = r'D:\DATA\g5ht-free\20251223'
FN = 'date-20251223_strain-ISg5HT_condition-starvedpatch_worm005'

f = h5py.File(os.path.join(PTH,FN + '.h5'), 'r')

im = f.get('img_nir')[:] # THW
sz = im.shape

fps = 20.0  # frames per second

print('Video shape: ', sz)

nframes = im.shape[0]
record_duration = nframes / fps # in seconds

print('Recording duration (s): ', record_duration)

data_dict = dict()
data_dict['input_h5'] = os.path.join(PTH,FN + '.h5')
data_dict['n_stack'] = 41
input_h5 = data_dict['input_h5']
n_stack = data_dict['n_stack']
new_dict = {}

print('keys:', list(f.keys()))

In [ ]:
print('keys:', list(f.keys()))

In [ ]:
f['daqmx_ai']

In [ ]:
plt.figure()
plt.plot(f['pos_st'])

In [ ]:
#FOR CONFOCAL (SIMPLE): confocal is on when daqmx_ai is high and pick when each stack starts
confocal_ons = (f['daqmx_di'][0] > 0.5).astype(int)
confocal_starts = np.where((np.diff(confocal_ons) == 1))[0]
confocal_stack_starts = confocal_starts[range(0, len(confocal_starts), n_stack)]

In [ ]:
f['daqmx_di']

In [ ]:
plt.figure()
plt.plot(f['daqmx_ai'][0][0:10000])
plt.plot(f['daqmx_ai'][1][0:10000])
plt.plot(f['daqmx_ai'][2][0:10000])
plt.show()

plt.figure()
plt.plot(f['daqmx_di'][0][0:10000])
plt.plot(f['daqmx_di'][1][0:10000])
plt.show()

In [ ]:
%matplotlib qt
plt.figure()
plt.plot(confocal_ons)
# overlay confocal_starts as red dots
plt.plot(confocal_starts, confocal_ons[confocal_starts], 'ro')
# overlay confocal_stack_starts as green dots
plt.plot(confocal_stack_starts, confocal_ons[confocal_stack_starts], 'go')
plt.show()

In [ ]:
confocal_starts

In [ ]:
#FOR NIR (COMPLICATED): nir is on when daqmx_di is high
nir_ons = (f['daqmx_di'][1] > 0.5).astype(int)
nir_starts = np.where((np.diff(nir_ons) == 1))[0]

#FOR NIR (COMPLICATED): i believe the camera only captures at certain nir ids
ids = np.array(f['img_metadata']['img_id'])
ids -= ids[0]

#FOR NIR (COMPLICATED): i believe the camera only adddionally saves every other frame
q = np.array(f['img_metadata']['q_iter_save']) > 0.5
filtered_nir_starts = nir_starts[ids[q]]

#combine the mappings
nir_to_conf_mapping = []
for i in confocal_stack_starts:
    index = np.argmax(filtered_nir_starts > i)
    if index > 0:
        nir_to_conf_mapping.append(index)
    else:
        break
new_dict['nir_to_conf_mapping'] = np.array(nir_to_conf_mapping)
data_dict.update(new_dict)

In [ ]:
import numpy as np
import importlib
import os

import nir.sync_nir_confocal_converted
_ = importlib.reload(sys.modules['nir.sync_nir_confocal_converted'])


# 1. Load and synchronize timing
PTH = r'D:\DATA\g5ht-free\20251223'
FN = 'date-20251223_strain-ISg5HT_condition-starvedpatch_worm005'
path_h5 = os.path.join(PTH, FN + '.h5')
confocal_to_nir, nir_to_confocal, timing_stack, timing_nir = nir.sync_nir_confocal_converted.sync_timing_from_file(
    path_h5, n_rec=1
)

print(f"Confocal frames: {len(confocal_to_nir)}")
print(f"NIR frames: {len(nir_to_confocal)}")

# 2. Setup complete timing information
data_dict = {}
param = {
    'n_rec': 1,
    'max_t': len(confocal_to_nir),
    't_range': range(len(confocal_to_nir)),
    'FLIR_FPS': 20.0
}

nir.sync_nir_confocal_converted.get_timing_info(data_dict, param, path_h5, h5_confocal_time_lag=0)

# 3. Align behavioral data
# Assume you have NIR-rate behavioral data
nir_velocity = np.random.randn(len(nir_to_confocal))  # example data
confocal_velocity = nir.sync_nir_confocal_converted.nir_vec_to_confocal(
    nir_velocity, 
    data_dict['confocal_to_nir'], 
    param['max_t']
)

# 4. Access synchronized timestamps
confocal_timestamps = data_dict['timestamps']
nir_timestamps = data_dict['nir_timestamps']

# 5. Find stimulus onsets
stim_frames_confocal = data_dict['stim_begin_confocal']
stim_frames_nir = data_dict['stim_begin_nir']

print(f"Stimulus delivered at confocal frames: {stim_frames_confocal}")

In [16]:
f['daqmx_di']

<HDF5 dataset "daqmx_di": shape (2, 3279948), type "<u4">

In [14]:
import h5py
import numpy as np
from scipy import signal
import tifffile as tif
import tqdm
import sys
import os.path as osp
import pandas as pd


PTH = r'D:\DATA\g5ht-free\20251028'
FN = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'
h5_path = osp.join(PTH, FN + '.h5')
n_stack = 41

#FOR CONFOCAL (SIMPLE): confocal is on when daqmx_ai is high and pick when each stack starts
f = h5py.File(h5_path, 'r')
confocal_piezo_ends, _ = signal.find_peaks(f['daqmx_ai'][1], distance=100, prominence=1)

#FOR NIR (COMPLICATED): nir is on when daqmx_di is high
nir_ons = (f['daqmx_di'][1] > 0.5).astype(int)
nir_starts = np.where((np.diff(nir_ons) == 1))[0]

#FOR NIR (COMPLICATED): i believe the camera only captures at certain nir ids
ids = np.array(f['img_metadata']['img_id'])
ids -= ids[0]

#FOR NIR (COMPLICATED): i believe the camera only adddionally saves every other frame
q = np.array(f['img_metadata']['q_iter_save']) > 0.5
filtered_nir_starts = nir_starts[ids[q]]

#combine the mappings
mapping = np.searchsorted(filtered_nir_starts, confocal_piezo_ends, side='right') - 1

#gets the filename
name = osp.basename(h5_path).split('.')[0]

#saves the nir
nir = np.zeros((len(mapping), 732, 968), dtype=np.ubyte)
for i, j in tqdm.tqdm(enumerate(mapping), total=len(mapping)):
    try:
        nir[i] = f['img_nir'][j]
    except Exception as e:
        break
tif.imwrite(f'{name}.tif', nir, imagej=True)

#saves the stage position
stage_data = np.array(f['pos_stage'])
df = pd.DataFrame(stage_data)
data = df.interpolate(method='cubic').to_numpy()[mapping]
y, x = data.T
pd.DataFrame(data).to_csv(f'{name}.csv')
print(f'{name} saved')

100%|██████████| 1200/1200 [00:00<00:00, 2828.51it/s]


date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001 saved


In [15]:
name

'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'

In [10]:
ids[q]

array([    0,     6,     8, ..., 26039, 26041, 26043], shape=(12826,))

In [9]:
nir_starts

array([    899,    1024,    1149, ..., 3253983, 3254108, 3257756],
      shape=(26027,))

## STARTING OVER ON SYNC

In [44]:
import h5py
import numpy as np
from scipy import signal
import tifffile as tif
import tqdm
import sys
import os.path as osp
import pandas as pd
import matplotlib.pyplot as plt


# PTH = r'D:\DATA\g5ht-free\20251028'
# FN = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'
PTH = r'D:\DATA\g5ht-free\20260123'
FN = 'date-20260123_strain-ISg5HT-nsIS180_condition-fedpatch_worm004'

h5_path = osp.join(PTH, FN + '.h5')
n_stack = 41
n_volume = 1200

In [45]:
f = h5py.File(h5_path, 'r')

In [46]:
ai_laser = f['daqmx_ai'][0]
ai_piezo = f['daqmx_ai'][1]
ai_stim = f['daqmx_ai'][2]
di_confocal = f['daqmx_di'][0]
di_nir = f['daqmx_di'][1]

img_timestamp = f['img_metadata']["img_timestamp"][:]
img_id = f['img_metadata']["img_id"][:]
q_iter_save = f['img_metadata']["q_iter_save"][:]



In [47]:
def filter_ai_laser(ai_laser: np.ndarray, di_camera: np.ndarray, n_rec: int = 1) -> np.ndarray:
    """
    Filter laser signal to include only z-stack recording periods.

    Parameters
    ----------
    ai_laser : np.ndarray
        Laser analog input
    di_camera : np.ndarray
        Camera digital input
    n_rec : int, optional
        Number of recordings to keep (default: 1)

    Returns
    -------
    np.ndarray
        Filtered laser signal
    """
    n = min(len(ai_laser), len(di_camera))
    ai_laser_zstack_only = ai_laser[:n].astype(np.float32)
    ai_laser_filter_bit = np.zeros(n, dtype=np.float32)
    trg_state = np.zeros(n, dtype=np.float64)

    # # validation plots
    # plt.figure(figsize=(12, 6))
    # plt.plot(ai_laser_zstack_only, label='Original AI Laser')
    # plt.plot(di_camera[:n], label='DI Camera', alpha=0.5)
    # plt.title('AI Laser and DI Camera Signals')
    # plt.xlabel('Sample Index')
    # plt.ylabel('Signal Amplitude')
    # plt.legend()
    # plt.show()

    n_kernel = 100
    for i in range(n):
        start = max(0, i - n_kernel)
        stop = min(n, i + n_kernel + 1)
        trg_state[i] = np.max(di_camera[start:stop])

    delta_trg_state = np.diff(trg_state)
    list_idx_start = np.where(delta_trg_state == 1)[0]
    list_idx_end = np.where(delta_trg_state == -1)[0]

    if n_rec > len(list_idx_start):
        raise ValueError("filter_ai_laser: not enough recordings detected. check n_rec")

    # Get n_rec longest recordings
    rec_lengths = list_idx_end - list_idx_start
    list_idx_rec = np.argsort(rec_lengths)[::-1][:n_rec]
    
    for i in list_idx_rec:
        ai_laser_filter_bit[list_idx_start[i]+1:list_idx_end[i]] = 1

    return ai_laser_filter_bit * ai_laser_zstack_only

ai_laser_filtered = filter_ai_laser(ai_laser, di_confocal, n_rec=1)

In [48]:
%matplotlib qt
ix2plot = slice(20000, 25000)
plt.figure()
plt.plot(ai_laser[ix2plot], label='Original AI Laser')
plt.plot(ai_laser_filtered[ix2plot], label='Filtered AI Laser', alpha=0.7)
# plt.plot(di_confocal[ix2plot], label='DI Confocal', alpha=0.5)
plt.title('AI Laser and Filtered AI Laser Signals')
plt.xlabel('Sample Index')
plt.ylabel('Signal Amplitude')
plt.legend()
plt.show()